# Welcome

Hello and welcome to this series of Jupyter Notebooks looking at The National Archives Discovery tool. 

There are two general aims with these notebooks:
-	Showing how to work with the Discovery API, and why you might want to use it. 
-	Looking into different things you might want to do with the data.

We start in this specific notebook by focusing on the Discovery API: what it is, how it differs from the web search, and potential benefits of using it. After this, the notebooks split into two paths: Named Entity Recognition, and Data visualisation. 


### Aims of this notebook

In this notebook, we are going to: 
-	Briefly introduce the concept of an API; explaining what one is and the benefits of using one. 
-	Have a look at the Discovery API and see what particular details we might expect to find. 
-	Work through a few examples of how to use and gather data for the following notebooks. 


## What is an API

It is best to start by expanding the acronym: an API is an Application Programming Interface. They are how customers (either you, or other programs) interact with a program. The API "sits in front of" the program itself, taking requests for things the program should do, and passing the instructions on to the program. 
APIs are extremely common, and many of them take human interaction. As an example, search engines all work with an API: you, as the human, provide the search term to the search app. The search app then converts this to an API query for its own database, adding in factors deemed relevant, such as prior search history or time of day.

There are a few different metaphors that help understand APIs and how they work. Here, we are going to use the Pet Store metaphor. One advantage to this metaphor is that the roles are straightforward to understand: the human user is the person walking into the store; the API itself is equivalent to everything you can do within the store; and the backend/database is everything “in the back” of the store. 

Over the next few steps, we are going to build an API request, with a look at each building block required.


#### Base path

The base path is the address - its where the API "is". In the pet store metaphor, this is equivalent to the physical address – you need to find the store itself before you can do anything with it. What this base path looks like will depend on where the API is located. For web-hosted API's the base path will look like a URL; if it is stored locally, it may be a file path; and if it is a database, it may be a database name. 

Discovery's API is web-hosted, so we will focus on URLs. 

In [ ]:
## The first building block we need is the base URL for the API we want to use. 
## For the pet shop metaphor, we'll use an example one, and set it to a variable - this way, we can use it later on in the notebook

pet_shop_base_url = "https://www.examplepetstore.com"

## we will setup the required data for the Discovery API later on in the notebook

#### Endpoints

The next building block is to specify which bit of the API you want to interact with - specified by endpoints. Once you've reached the petstore (API), you need to decide whether you want to interact with a specific pet, the inventory, or the employees. In simpler APIs there might just be one or two endpoints, in more complex APIs there will be several. 

Different APIs will be designed to work in different ways: if the store has a very mixed stock, it may make more sense to only have a single generic endpoint for individual pets; if the store specialises in cats, it may be appropriate to have an endpoint dedicated to cats. Often, these design choices will be made with the user, the query parameters, and the data in mind. For example, having the cats split off will allow for a more useful and relevant set of query paramaters to be specified, compared to a generic endpoint requiring filters applicable to all pets.

In [ ]:
## For the pet shop, we might want to interact with an individual pet

pet_endpoint = "/pet"

## If we stick the base url and this endpoint together, we get the full url for the pet endpoint - https://www.examplepetstore.com/pet

## We might also want to interact with a few other endpoints:
### the pet store's inventory of pets

inventory_endpoint = "/inventory"

### the pet store's employees

employees_endpoint = "/employees"

### the pet store's customers

customers_endpoint = "/customers"

#### Variables within the URL

When constructing a URL, you can add a variable in it. This can help if, for example, the pet store has multiple branches, and you want information from a specific one. This means you can provide a single API, but have a much broader range of information available. In documentation, this is often shown as a curly bracketed variable, such as `{branch}`.

In [ ]:
## Lets assume we want information about the store in Leicester

specific_store = "leicester"                # feel free to change this to another store, and see what happens

## If the documentation shows that the endpoint /store/{store_name} is available, we can use the .format() method to insert the specific store name into the endpoint
## This will input the specific store name neatly into the endpoint. 

specific_store_endpoint = "/store/{}".format(specific_store)

## To see what this looks like, we can print the endpoint

print(specific_store_endpoint)

#### Query parameters

Query parameters are used to specify precisely what information you want to interact with. Often, they are useful if you do not yet know/do not want to interact with a specific ID; instead wanting to search or interact across the records avaialble via the API. If, for example, you wanted to find all ginger cats, you would likely use a query parameter to specify this. 

There are a few formatting detials worth highlighting for query paramaters. As they are filters for a specific query, they are always added to the end of the URL, after a question mark. They are always in the format key=value, where the key is the specific filter you want to use, and the value is what you want to filter by. If you want to use multiple query parameters, you can use an ampersand (&) to separate them.

In [ ]:
## Continuing with the Leicester store example, we might want to get information about the store's cats

## We start by specifying that we want to interact with the inventory endpoint - remember, we are searching for potentially several cats, rather than interacting with a specific cat

specific_store_inventory_endpoint = "/store/{}/inventory".format(specific_store)

## Lets assume that we want to filter for cats, and the colour ginger. We will set these up as a dictionary.
specific_store_cats_filters = {
    "type": "cat",
    "colour": "ginger"
}

## We can then append these to the endpoint. We will use a small for loop to do this

specific_store_inventory_endpoint += "?"                                 ## Start by adding a question mark to the end of the endpoint

for key, value in specific_store_cats_filters.items():                   ## Run through every item in the dictionary, and for each extract the key and value
    specific_store_inventory_endpoint += "{}={}".format(key, value)      ## Append the key and value to the endpoint. For .format(), if multiple values are passed, they are inserted in the order they appear
    if key != list(specific_store_cats_filters.keys())[-1]:              ## If the key is not the last key in the dictionary, add an ampersand to the end of the endpoint
        specific_store_inventory_endpoint += "&"

## We can then print the endpoint to see what it looks like

print(specific_store_inventory_endpoint)

##### Reqest methods

Ther are a few different “methods” that can be used to interact with an API. In this case, the word method is being used to describe the type of request – describing what you would like to do with the database in the backend, rather than how you are sending the request. 

Using a different method with the same url and query parameters will return different results. For example, using a GET request will return the information about the pet, whereas a DELETE request will remove the pet from the database.

Commonly used methods are (note that they are typically displayed in allcaps): 
-	GET – I would like the information you have on this thing (e.g. I would like information about this specific pet)

e.g. To get all the information about the pet with ID 1, you would use the following request:

```
method: GET
url: https://petstore.com/pets/1
```


-   POST – I would like to add new information about this thing (e.g. I would like to add a new pet to the store, or add an image to a specific pet). 

e.g. To add a new pet to the store, with the ID 1. As this is a new pet, the store would not have any information, and the customer would have to provide it as part of the request payload. 
```
method: POST
url: https://petstore.com/pets/1
```

-   PUT – I would like to update the information about this thing (e.g. This pet has had a birthday and I would like to update its age). 

e.g. To update the information about the pet with ID 1. In this case, the body of the request would include the new information.

```
method: PUT
url: https://petstore.com/pets/1
```

-   DELETE – I would like to remove this information (e.g. A pet has been bought and its records need to be removed from the store).

e.g. To remove the information about the pet with ID 1.
```
method: DELETE
url: https://petstore.com/pets/1
```

#### Response codes

Response codes are the different responses that you can get from the API, and are worth noting. In the pet store, this would be the different responses you can get from the store - the pet you wanted is in stock, the pet you wanted is not in stock, the pet you wanted is in stock but is reserved, etc. Typically, there are a few different response codes that you will see:

- 200 - OK - the request was successful
- 400 - Bad Request - the request was not successful
- 401 - Unauthorized - the request was not successful because you are not authorized to make that request
- 403 - Forbidden - the request was not successful because you are not allowed to make that request
- 404 - Not Found - the request was not successful because the thing you are looking for does not exist
- 500 - Internal Server Error - the request was not successful because there was an error on the server

#### Response

The response is the data that you actually get back from the API. It can come in a few different formats, but the most common are JSON and XML. 



## What is Discovery

[Discovery](https://discovery.nationalarchives.gov.uk/) is the catalogue for [The National Archives](https://www.nationalarchives.gov.uk/). It contains descriptions of over 32 million records held by The National Archives and more than 2,500 archives across the country. Over 9 million records are available for download. 

Powering Discovery is a database; it is this database that stores the information about each record and is queried every time (either by the web user interface, or the API). It is the sheer number of records in Discovery that make using the API a valuable option. Rather than having to use the web interface separately for each unique query, following the rest of this notebook you will be able to automate its use - allowing you to filter more efficiently and retrieve large amounts of data more effectively. 

## Gathering information
As we saw above, the key thing you need when working with an API is the documentation. This provides information on what requests you can build: what endpoints are available, what query parameters you can use, what methods they accept, and so on. This information is often provided either directly alongside, or is similarly accessible as, the API. 

To start finding information on Discovery, searching the internet for 
 "Discovery API documentation" will bring up the [main help page](https://www.nationalarchives.gov.uk/help/discovery-for-developers-about-the-application-programming-interface-api/). Some information is available here - particularly the terms of use and information on catalogue levels. From this page, you can find the [API sandbox](https://discovery.nationalarchives.gov.uk/API/sandbox/index). This page provides you with an interactive way to test out the API, and see what data you can retrieve. It also works as documentation. 

At this stage, try it out! Have a go with the sandbox and see what you can retrieve.

Three key bits of information you can gather from the Sandbox are:
- The query method
- The base URL
- The parameters, and which ones are required vs optional

In [ ]:
base_discovery_url = "https://discovery.nationalarchives.gov.uk/API"

search_endpoint = "/search/records"

### What can we find out
The first thing we find wen looking into the sandbox is the number of different paths/endpoints available. This represents the variety of works typically undertaken with the Archives. You may be looking for the authority about a record, or you may be looking for details about a record with a specific identification. If we investigate these endpoints closer, we can see that they all require an identification number for the record(s) being investigated. 

For the purpose of this Notebook, we are going to stick to searching for records. So, we will stick to the very last endpoint in the list: `/search/records`. The documentation shows us that this endpoint will accept GET requests only, logical for a search API. 

When expanded, we can see that there are a lot of parameters we can see there are a considerable range of parameters available to use. This is an effect of two things: 
- The range of data available on Discovery
- The details available about each record.  

Helpfully for the user, only one of these parameters is required: the search term. 

We will start by building a list of all the optional parameters, and marking each one with a true/false value depending on whether we intend to use it. For simplicity, we are marking them all as false for this first example. Note that search query is not on this list - it is required, and so will be added to the request regardless.


In [ ]:
## We will set up the list as a set of dictionaries, with each having a name and a true/false value
## for whether we are going to use it.
## Note: we are not going to include all the parameters, as there are a lot available, and with 3 different
## bits of information for each, it would take up a lot of space. We will include a few, and you can
## add more if you want to.

search_parameters = [
    {
        name = "record series",
        discovery_name = "sps.recordSeries",
        use = False
    },
    {
        name = "date from",
        discovery_name = "sps.dateFrom",
        use = False
    },
    {
        name = "date to",
        discovery_name = "sps.dateTo",
        use = False
    },
    {
        name = "time periods",
        discovery_name = "sps.timePeriods",
        use = False
    },
    {
        name = "record repositories",
        discovery_name = "sps.recordRepositories",
        use = False
    },
    {
        name = "departments",
        discovery_name = "sps.departments",
        use = False
    },
    {
        name = "taxonomy subjects",
        discovery_name = "sps.taxonomySubjects",
        use = False
    },
    {
        name = "catalougue levels",
        discovery_name = "sps.catalogueLevels",
        use = False
    },
    {
        name = "closure statuses",
        discovery_name = "sps.closureStatuses",
        use = False
    }
    {
        name = "held by code",
        discovery_name = "sps.heldByCode",
        use = False
    },
    {
        name = "document type",
        discovery_name = "sps.documentType",
        use = False
    }
]

## Building a query
The tool we are going to use to build requests is called [Requests](https://requests.readthedocs.io/en/master/). This is a Python library that allows you to send HTTP requests - and is equivalent to talking to the waiter in the restaurant. This is a widely used library - we will go over details here, but there is plenty of documentation available online. The first thing we need to do is import the library. Note: we are going to assume you do not have the library installed, and so will install it here. In real code, you do not need to include the installation step.

In [ ]:
%pip install requests
import requests

### Simplified starter look at requests

A request requires a URL, method, and parameters. 
The URL is the base URL, plus the endpoint. 
The method is the type of request - in this case, we are going to use GET. 
The parameters are the parameters we want to use.

The reqest library takes these three things and combines them into a request. It then sends this request to the API, and returns the response.

## Running a query
Now the various bits of the query have been defined, we can put them together and run the query. When using the reqesuts library, this is very straightforwards, and can be done in a single line of code. 
This line of code gathers together the base URL, the endpoint, and the parameters, and sends them to the API. The API then sends back a response, which is stored in the variable called "response". Here, the response is printed out - this lets us see what the response looks like. In reality, printing is a good option while testing, and may be useful for real use. There are, however, other things you may want to do. We will look at one of these later on.

## Making the results more readable

Looking at the "default" response we recieved from Discovery, we can see that there is a lot of data for each record - it will probably be useful to reduce this down. As the data returned from discovery is a JSON file, we can use the JSON library to make this easier.
Demonstrated here is a way to filter the response down to just the Context, Reference Number, and Held-by data. 
To start, a new dictionary is created. This will be used to store the data we want to keep.
We then loop through the response, and for each record we select the data we want to keep. This is then added to the new dictionary.
Finally, we print out the new dictionary. This is much easier to read than the default response.

## Now what?
- Getting digitised resources
- How to deal with non-digitised resources
- Options in the subsequent notebooks. 

One of the limitations of the Discovery API is that it can only recover data in the catalogue itself - this is the metadata about the record, and not the record itself. There are diff

##### Digitised records

Some records in Discovery have been digitised. This information is not yet avaiable in the Discovery API, but we can use the {number} parameter to check if it is avaiable. 

##### Non-digitised records

For non digitised records, the only approach is to go to The National Archives and look at the record in person. Instructions for how to do this can be found [here](https://www.nationalarchives.gov.uk/help-with-your-research/visit-us/) - note in particular that some records need to be ordered in advance to ensure they are avaiable when you visit. 

##### Options in the subsequent notebooks

Now that we have searched Discovery for some data and retreived it - the next notebooks start to do things with them. There are two paths that can be followed - Named Entity Recognition, and Mapping and Visualisation. 

##### Named Entity Recognition (NER)

##### Mapping and Visualisation

Here, the notebooks take you through the process of visualising data retrived from ships logs. The general aim of this series of notebooks is to take the data we can gather from the Discovery API, and show how we can work with it, to provide additional visual context and information.

